https://mp.weixin.qq.com/s?__biz=MjM5ODkzMzMwMQ==&mid=2650411643&idx=3&sn=5f7b6a07295a54d5ea3d1b453f182dd5&chksm=becd942189ba1d37eb117055b157f7453f6f9480d34e4fa76348cc0501be559a62b6978f83c1&scene=0&xtrack=1&key=e8c75d77299604ef2a6d47918d6c3921ebde89c40923818bd02987322d5a07b604c7efa701c62098d9fba9f0834fc790be3dde51f32d8518d6d25488ee65589079fa2b65f7a1e9e6e850659aaff3eb2d&ascene=1&uin=MjA1MjAyODkxNg%3D%3D&devicetype=Windows+10&version=62070141&lang=zh_CN&pass_ticket=%2FljxmLK3OdGy6PUNNDKy%2FPUNR5POGV%2B2r7xuqNUwneZfVrH0p0KbDlMznJxrHhgS

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [3]:
train = pd.read_csv('../dataset/test_csv/train.csv').fillna(' ')
test = pd.read_csv('../dataset/test_csv/test.csv').fillna(' ')

In [4]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [6]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text,test_text])

In [7]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True, 
    strip_accents='unicode', 
    analyzer='word',
    token_pattern=r'\w{1,}', 
    stop_words='english', 
    ngram_range=(1, 1), 
    max_features=10000
)

In [8]:
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [11]:
# 为了充分表征文本信息，也提取文本字的ngram信息
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2,6),
    max_features=50000
)

In [12]:
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [13]:
train_features = hstack([train_char_features,train_word_features])
test_features = hstack([test_char_features,test_word_features])

In [14]:
scores = []

In [15]:
submission = pd.DataFrame.from_dict({'id':test['id']})

In [17]:
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1,solver='sag')
    
    cv_score = np.mean(cross_val_score(classifier,train_features,train_target,cv=3,scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name,cv_score))
    
    classifier.fit(train_features,train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:,1]
print('Total CV score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9692156785561917
CV score for class severe_toxic is 0.9875939900087136
CV score for class obscene is 0.9838699747433383
CV score for class threat is 0.9833766978914316
CV score for class insult is 0.9774264477092549
CV score for class identity_hate is 0.9739425888301616
Total CV score is 0.9792375629565154


In [18]:
from sklearn.metrics import roc_auc_score

In [19]:
test_label = pd.read_csv('../dataset/test_csv/test_labels.csv')

In [24]:
test_label.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [21]:
auc_sum = 0

In [27]:
for class_ in class_names:
    print(test_label.id.isin(test_label[test_label[class_] == -1].id.tolist()))
    sub_test_label = test_label[test_label.id.isin(test_label[test_label[class_] == -1].id.tolist()) == False]
    print(len(sub_test_label))

0         True
1         True
2         True
3         True
4         True
          ... 
153159    True
153160    True
153161    True
153162    True
153163    True
Name: id, Length: 153164, dtype: bool
63978
0         True
1         True
2         True
3         True
4         True
          ... 
153159    True
153160    True
153161    True
153162    True
153163    True
Name: id, Length: 153164, dtype: bool
63978
0         True
1         True
2         True
3         True
4         True
          ... 
153159    True
153160    True
153161    True
153162    True
153163    True
Name: id, Length: 153164, dtype: bool
63978
0         True
1         True
2         True
3         True
4         True
          ... 
153159    True
153160    True
153161    True
153162    True
153163    True
Name: id, Length: 153164, dtype: bool
63978
0         True
1         True
2         True
3         True
4         True
          ... 
153159    True
153160    True
153161    True
153162    True
153163    True